In [1]:
!pip install ydata-profiling

import pandas as pd
from ydata_profiling import ProfileReport
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.2/356.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.7 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=5bf684efe34842888602ed4833867b51a85e8cc576993bb64ac21ecbac9d3381
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
df = pd.read_csv('/content/drive/MyDrive/MACHINE LEARNING/3. Zjazd/Warsztat projektowy/data.csv')

In [4]:
df.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [5]:
profile = ProfileReport(df, title="News Dataset Report")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
politics_articles = df[df['Headline'].str.contains('politics', case=False, na=False)]
print(f"Number of articles with word 'politics' in Headline: {len(politics_articles)}")

Number of articles with word 'politics' in Headline: 9


In [7]:
df = df.drop(columns=['URLs'])
df = df.dropna()
df = df.drop_duplicates()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3522 entries, 0 to 4008
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  3522 non-null   object
 1   Body      3522 non-null   object
 2   Label     3522 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 110.1+ KB


In [9]:
df['text'] = df['Headline'] + ' ' + df['Body']

In [10]:
df.head()

,Headline,Body,Label,text
0,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1,Four ways Bob Corker skewered Donald Trump Ima...
1,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1,Linklater's war veteran comedy speaks to moder...
2,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1,Trump’s Fight With Corker Jeopardizes His Legi...
3,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1,Egypt's Cheiron wins tie-up with Pemex for Mex...
4,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1,Jason Aldean opens 'SNL' with Vegas tribute Co...


In [11]:
# Normalization of text
df['text'] = df['text'].str.lower()

def clean_tweet(tweet):
   tweet = re.sub(r'source\s*www\S+', '', tweet)
   tweet = re.sub(r'source\s*http\S+', '', tweet)
   tweet = re.sub(r'http\S+|www\S+|https\S+', '<URL>', tweet, flags=re.MULTILINE)  # Deleting URLs
   tweet = re.sub(r'@\w+', '<USER>', tweet)   # Deleting mentions of users
   tweet = re.sub(r'#', '<TAG>', tweet) # Deleting hashtags, preserving text
   tweet = re.sub(r'\d+', '<NUMBER>', tweet)   # Deleting digits
   return tweet

df['text'] = df['text'].apply(clean_tweet)

# Deleting numbers, special digits and additional spaces
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'\W+', ' ', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Deleting stopwords
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Lemmatization
lemmatizer = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [12]:
df.iloc[5]['Body']

"JetNation FanDuel League; Week 4\n% of readers think this story is Fact. Add your two cents.\n(Before It's News)\nOur FanDuel league is back again this week. Here are the details:\n$900 in total prize money. $250 to the winner. $10 to enter.\nRemember this is a one week league, pick your lineup against the salary cap and next week if you want to play again you can pick a completely different lineup if you want.\nClick this link to enter — http://fanduel.com/JetNation\nYou can discuss this with other NY Jets fans on the Jet Nation message board. Or visit of on Facebook.\nSource: http://www.jetnation.com/2017/09/27/jetnation-fanduel-league-week-4/"

In [13]:
df.iloc[5]['text']

'jetnation fanduel league week NUMBER jetnation fanduel league week NUMBER reader think story fact add two cent news fanduel league back week detail NUMBER total prize money NUMBER winner NUMBER enter remember one week league pick lineup salary cap next week want play pick completely different lineup want click link enter URL discus ny jet fan jet nation message board visit facebook source URL'

In [15]:
df['Label'].value_counts()

,count
Label,
0,1803
1,1719


In [16]:
df[df['Label']==0]

,Headline,Body,Label,text
5,JetNation FanDuel League; Week 4,JetNation FanDuel League; Week 4\n% of readers...,0,jetnation fanduel league week NUMBER jetnation...
10,U.S. President Donald Trump Quietly Signs Law ...,By Aaron Kesel\nAs former White House chief of...,0,u president donald trump quietly sign law allo...
11,2017 Fantasy Football Team Defense Rankings - ...,2017 Fantasy Football Team Defense Rankings – ...,0,NUMBER fantasy football team defense ranking w...
12,Just Shut Up & Play Some Damn Baseball!!,Just Shut Up & Play Some Damn Baseball!!\n(Bef...,0,shut play damn baseball shut play damn basebal...
14,"10/7: Chuck Axed; HBD Brickyard, Adam, Moonlig...",A Potato Battery Can Light up a Room for Over ...,0,NUMBER NUMBER chuck axed hbd brickyard adam mo...
...,...,...,...,...
3998,"9/28 Through the 40s: The Gloaming; HBD Bill, ...","No Getting Around it, The War Is Coming! Trump...",0,NUMBER NUMBER NUMBER gloaming hbd bill lou cy ...
4000,35 False Matches and 1 Erroneous Arrest As Pol...,By Nicholas West\nAll things biometric are swe...,0,NUMBER false match NUMBER erroneous arrest pol...
4003,CNN and Globalist Exposed - Steve Quayle and A...,"Vietnam Is in Great Danger, You Must Publish a...",0,cnn globalist exposed steve quayle alex jones ...
4005,Trump Jr. Is Soon To Give A 30-Minute Speech F...,Trump Jr. Is Soon To Give A 30-Minute Speech F...,0,trump jr soon give NUMBER minute speech NUMBER...


In [17]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np

In [18]:
df['text_tokenized'] = df['text'].apply(lambda x: x.split())

# Word2Vec training
w2v_model = Word2Vec(sentences=df['text_tokenized'], vector_size=200, window=10, min_count=2, sg=0, epochs=150)

In [19]:
def get_average_w2v(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Counting Word2Vec vectors for each article
df['w2v_vector'] = df['text_tokenized'].apply(lambda x: get_average_w2v(x, w2v_model))

# Conversion from list of vectors to columns
X = np.array(df['w2v_vector'].tolist())
y = df['Label']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Logistic Regression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
print("Logistic Regression:")
print(classification_report(y_test, y_pred_lr))

# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))

# Support Vector Machine
svm_model = SVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
print("Support Vector Machine:")
print(classification_report(y_test, y_pred_svm))

Logistic Regression:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       363
           1       0.96      0.96      0.96       342

    accuracy                           0.96       705
   macro avg       0.96      0.96      0.96       705
weighted avg       0.96      0.96      0.96       705

Random Forest:
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       363
           1       0.91      0.98      0.94       342

    accuracy                           0.94       705
   macro avg       0.94      0.94      0.94       705
weighted avg       0.94      0.94      0.94       705

Support Vector Machine:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       363
           1       0.97      0.99      0.98       342

    accuracy                           0.98       705
   macro avg       0.98      0.98      0.98       705
weighted avg  

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
# GridSearch for Logistic Regression
param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs']
}

# GridSearch for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# GridSearch for SVM
param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

In [24]:
# Logistic Regression optimalization
grid_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=5, scoring='precision')
grid_lr.fit(X_train, y_train)
best_params_lr = grid_lr.best_params_

# Random Forest optimalization
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='precision')
grid_rf.fit(X_train, y_train)
best_params_rf = grid_rf.best_params_

# SVM optimalization
grid_svm = GridSearchCV(SVC(), param_grid_svm, cv=5, scoring='precision')
grid_svm.fit(X_train, y_train)
best_params_svm = grid_svm.best_params_

print("Best parameters for Logistic Regression:", best_params_lr)
print("Best parameters for Random Forest:", best_params_rf)
print("Best parameters for SVM:", best_params_svm)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

Najlepsze parametry dla Logistic Regression: {'C': 1, 'solver': 'liblinear'}
Najlepsze parametry dla Random Forest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Najlepsze parametry dla SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


Best parameters for Logistic Regression: {'C': 1, 'solver': 'liblinear'}

Best parameters for Random Forest: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}

Best parameters for SVM: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}

In [29]:
# Logistic Regression with best parameters
lr_model_opt = LogisticRegression(**best_params_lr)
lr_model_opt.fit(X_train, y_train)
y_pred_lr_opt = lr_model_opt.predict(X_test)

# Random Forest with best parameters
rf_model_opt = RandomForestClassifier(**best_params_rf)
rf_model_opt.fit(X_train, y_train)
y_pred_rf_opt = rf_model_opt.predict(X_test)

# SVM with best parameters
svm_model_opt = SVC(**best_params_svm)
svm_model_opt.fit(X_train, y_train)
y_pred_svm_opt = svm_model_opt.predict(X_test)

In [30]:
print("---------------- Logistic Regression ----------------")
print(classification_report(y_test, y_pred_lr))

print("- Logistic Regression (optimalized hyperparameters) -")
print(classification_report(y_test, y_pred_lr_opt))

---------------- Logistic Regression ----------------
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       363
           1       0.96      0.96      0.96       342

    accuracy                           0.96       705
   macro avg       0.96      0.96      0.96       705
weighted avg       0.96      0.96      0.96       705

- Logistic Regression (optimalized hyperparameters) -
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       363
           1       0.96      0.96      0.96       342

    accuracy                           0.96       705
   macro avg       0.96      0.96      0.96       705
weighted avg       0.96      0.96      0.96       705



In [31]:
print("------------------- Random Forest -------------------")
print(classification_report(y_test, y_pred_rf))

print("--- Random Forest (optimalized hyperparameters) ---")
print(classification_report(y_test, y_pred_rf_opt))

------------------- Random Forest -------------------
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       363
           1       0.91      0.98      0.94       342

    accuracy                           0.94       705
   macro avg       0.94      0.94      0.94       705
weighted avg       0.94      0.94      0.94       705

--- Random Forest (optimalized hyperparameters) ---
              precision    recall  f1-score   support

           0       0.95      0.92      0.94       363
           1       0.92      0.95      0.94       342

    accuracy                           0.94       705
   macro avg       0.94      0.94      0.94       705
weighted avg       0.94      0.94      0.94       705



In [32]:
print("-------------- Support Vector Machine  --------------")
print(classification_report(y_test, y_pred_svm))

print("-------- SVM (optimalized hyperparameters) --------")
print(classification_report(y_test, y_pred_svm_opt))

-------------- Support Vector Machine  --------------
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       363
           1       0.97      0.99      0.98       342

    accuracy                           0.98       705
   macro avg       0.98      0.98      0.98       705
weighted avg       0.98      0.98      0.98       705

-------- SVM (optimalized hyperparameters) --------
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       363
           1       0.96      0.98      0.97       342

    accuracy                           0.97       705
   macro avg       0.97      0.97      0.97       705
weighted avg       0.97      0.97      0.97       705

